In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Analisi modale del Lunar Lander

La matrice dinamica $A$ che rappresenta le dinamiche del Lunar Lander (vedi l'esempio [dinamiche di posizione laterale del lander lunare](SS-11-Lunar_lander_lateral_position_dynamics) per maggiori dettagli) è:

$$
A=\begin{bmatrix}0&1&0&0 \\ 0&0&F/m&0 \\ 0&0&0&1 \\ 0&0&0&0\end{bmatrix},
$$

dove $F$ è la forza di propulsione e $m$ la massa del lander. Lo stato del sistema è $x=[z,\dot{z},\theta,\dot{\theta}]^T$, dove $z$ è la posizione laterale, $\dot{z}$ la velocità laterale, $\theta$ l'angolo di orientamento del lander rispetto alla verticale e $\dot{\theta}$ la sua variazione nel tempo.

La matrice dinamica in questa forma mostra quattro autovalori, tutti uguali a 0. Gli autovalori nulli sono spesso chiamati integratori (si ricordi la Trasformata di Laplace dell'integrale di un segnale: qual è la radice del denominatore della sua espressione?). Quindi, questo sistema si dice che ha 4 integratori. Con $F\neq0$ ($m\neq0$) il sistema presenta una struttura simile ad un blocco di Jordan $4\times4$, quindi l'autovalore 0, in questo caso, ha una molteplicità geometrica uguale a 1. Con $F=0$ l'autovalore rimane lo stesso con la stessa molteplicità algebrica ma con una molteplicità geometrica pari a 2.

Di seguito è presentato un esempio con $F\neq0$.

### Come utilizzare questo notebook?

- Prova a impostare $F=0$ e prova a spiegare cosa implica fisicamente questo caso per il lander, specialmente per le dinamiche $z$ e $\theta$ e la loro relazione.

In [2]:
#Preparatory Cell 
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define the sliders for m, k and c
m = widgets.FloatSlider(
    value=1000,
    min=400,
    max=2000,
    step=1,
    description='$m$ [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
F = widgets.FloatSlider(
    value=1500,
    min=0,
    max=5000,
    step=10,
    description='$F$ [N]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
#function that make all the computations
def main_callback(m, F):
    eig1 = 0
    eig2 = 0
    eig3 = 0
    eig4 = 0
    
    if numpy.real([eig1,eig2,eig3,eig4])[0] == 0 and numpy.real([eig1,eig2,eig3,eig4])[1] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig1,eig2,eig3,eig4]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig1,eig2,eig3,eig4]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig1,eig2,eig3,eig4]))),1000)
    if F==0:
        mode1 = numpy.exp(eig1*T)
        mode2 = T*mode1
        mode3 = mode1
        mode4 = mode2
    else:
        mode1 = numpy.exp(eig1*T)
        mode2 = T*mode1
        mode3 = T*mode2
        mode4 = T*mode3
    
    fig = plt.figure(figsize=[16, 10])
    fig.set_label('Modi')
    g1 = fig.add_subplot(221)
    g2 = fig.add_subplot(222)
    g3 = fig.add_subplot(223)
    g4 = fig.add_subplot(224)
    
    g1.plot(T,mode1)
    g1.grid()
    g1.set_xlabel('t [s]')
    g1.set_ylabel('Primo modo')
    
    g2.plot(T,mode2)
    g2.grid()
    g2.set_xlabel('t [s]')
    g2.set_ylabel('Secondo modo')
    
    g3.plot(T,mode3)
    g3.grid()
    g3.set_xlabel('t [s]')
    g3.set_ylabel('Terzo modo')
    
    g4.plot(T,mode4)
    g4.grid()
    g4.set_xlabel('t [s]')
    g4.set_ylabel('Quarto modo')
    
    modesString = r'L\'unico autovalore del sistema è 0 con molteplicità algebrica 4. '
    if F==0:
        modesString = modesString + r'I modi corrispondenti sono $k$ e $t$.'
    else:
        modesString = modesString + r'I modi corrispondenti sono $k$, $t$, $\frac{t^2}{2}$ e $\frac{t^3}{6}$.'
    display(Markdown(modesString))

        
out = widgets.interactive_output(main_callback,{'m':m,'F':F})
sliders = widgets.HBox([m,F])
display(out,sliders)

Output()